In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo_codes import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
#miles_conv = 0.000621371

In [6]:
#from datetime import datetime
#now = datetime.now()

In [7]:
# define file path and read raw data for nyc origin cbgs
path='/Users/Dara/Documents/Python/rp-safegraph/'
cbg_nyc = pd.read_csv(path +"nyc_cbg.csv")

In [8]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [9]:
#define the months and days
#monthList =["01","02","03","04","05","06", "07"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [10]:
##April 2020
frames = []
for i in range(0,3):
    with fs.open(f'sg-c19-response/social-distancing/v2/2020/04/{dayList[i]}/2020-04-{dayList[i]}-social-distancing.csv.gz','rb') as f:
        print(f'04-{dayList[i]}')
        # read SG's file
        df = pd.read_csv(f, escapechar='\\', compression='gzip')
        # filter NYC's Origin CBGs
        dff = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")
        frames.append(dff)

04-01
04-02
04-03


In [11]:
#concat final dataframe and get rid of null cbgs from safegraph set
df = pd.concat(frames)
#df.head()

In [12]:
df.head()

,Pop_10E,orig_st,orig_co,orig_cbg,orig_stco,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home
0,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}"
1,1120,36,5,360050002001,36005,360050002001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,48,6235.0,"{""0"":28,"">50000"":3,""2001-8000"":1,""1-1000"":5,""1...","{"">50000"":11,""<1000"":15,""2001-8000"":12,""1001-2...",26,1009,"{""721-1080"":12,""361-720"":2,""61-360"":4,""<60"":8,...","[26,29,30,28,29,25,26,29,30,32,30,30,31,28,26,...",1,1,"{""361190031002"":1,""360050044004"":2,""3600500420...",1,0,82,"{""21-45"":4,""46-60"":1,""721-840"":1,""<20"":30,""61-...",100,"{""26-50"":1,""51-75"":3,""0-25"":11,"">100"":4,""76-10..."
2,1974,36,5,360050002002,36005,360050002002,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,82,3437.0,"{""16001-50000"":1,""0"":43,""2001-8000"":14,""1-1000...","{""16001-50000"":5,""<1000"":92,""2001-8000"":155,""1...",43,1003,"{""721-1080"":13,""361-720"":5,""61-360"":9,""<60"":10...","[52,52,54,55,53,52,54,51,50,50,45,47,46,45,44,...",1,1,"{""360050173004"":1,""340030322023"":1,""3600502010...",1,0,189,"{""21-45"":4,""481-540"":1,""541-600"":1,""46-60"":2,""...",100,"{""26-50"":3,""51-75"":5,""0-25"":8,"">100"":2,""76-100..."
3,1240,36,5,360050002003,36005,360050002003,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,57,2320.0,"{""16001-50000"":1,""0"":27,"">50000"":1,""2001-8000""...","{""16001-50000"":218,"">50000"":47,""<1000"":300,""20...",29,927,"{""721-1080"":5,""361-720"":1,""61-360"":10,""<60"":9,...","[32,35,34,37,35,35,33,31,30,29,29,30,33,29,30,...",1,1,"{""340030234021"":1,""360610014012"":1,""3600500190...",3,0,105,"{""21-45"":1,""481-540"":1,""721-840"":1,""<20"":32,""6...",100,"{""26-50"":2,""51-75"":2,""0-25"":11,"">100"":2,""76-10..."
4,0,36,5,360050004000,36005,360050004000,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,18,2022.0,"{""0"":4,"">50000"":1,""2001-8000"":4,""1-1000"":3,""10...","{"">50000"":247,""<1000"":444,""2001-8000"":189,""100...",4,0,"{"">1080"":1,""<60"":11,""361-720"":1,""61-360"":3}","[9,4,5,4,3,2,2,1,4,1,1,1,1,3,1,2,1,1,1,2,2,1,2,1]",1,1,"{""421010390001"":1,""360050086002"":2,""3606100220...",1,182,54,"{""21-45"":2,""541-600"":2,""1201-1320"":1,""301-360""...",0,"{"">100"":1,""0-25"":9,""76-100"":2,""26-50"":1}"


In [13]:
#unpack destination cbg column
df = sgpy.unpack_json_and_merge(df, json_column='bucketed_away_from_home_time', key_col_name='away_from_home_time', value_col_name='time_away_count')

In [22]:
#Make new columns for data analysis & aggregation
df['orig_cbg'] = df['orig_cbg'].apply(str)
df['orig_tract'] = df['orig_cbg'].str[:-1]
#df['is_home'] = df.apply(lambda x: x['orig_cbg']==x['destination_cbg'],axis=1)
df['date_y-m-d'] = df['date_range_start'].str[:10]
#df['dest_stco_all'] = df['destination_cbg'].str[:5]
#df['dest_stco_reg'] = df['dest_stco_all']
#df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR'
#df['dest_sub'] = df['dest_stco_all'].map(sub).fillna('O31CR')
df.head()

,Pop_10E,orig_st,orig_co,orig_cbg,orig_stco,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,away_from_home_time,time_away_count,date_y-m-d,orig_tract
0,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}",481-540,1,2020-04-01,36005000100
1,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}",1201-1320,1,2020-04-01,36005000100
2,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}",301-360,1,2020-04-01,36005000100
3,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}",61-120,1,2020-04-01,36005000100
4,11091,36,5,360050001001,36005,360050001001,2020-04-01T00:00:00-04:00,2020-04-02T00:00:00-04:00,10,14342.0,"{""16001-50000"":1,"">50000"":1,""2001-8000"":1,""1-1...","{""16001-50000"":171,"">50000"":582,""<1000"":234,""2...",1,0,"{""<60"":6,""61-360"":1}","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0]",1,1,"{""360870106011"":1,""360810031002"":1,""3610319040...",1,304,40,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""61-120...",0,"{""0-25"":6,""51-75"":1}",121-180,5,2020-04-01,36005000100


In [23]:
#select only the columns we want to analyze
df_away = df[['orig_tract','away_from_home_time','time_away_count']]

In [24]:
df_away.head(20)

,orig_tract,away_from_home_time,time_away_count
0,36005000100,481-540,1
1,36005000100,1201-1320,1
2,36005000100,301-360,1
3,36005000100,61-120,1
4,36005000100,121-180,5
5,36005000100,421-480,1
6,36005000200,21-45,4
7,36005000200,46-60,1
8,36005000200,721-840,1
9,36005000200,<20,30


In [25]:
dff_away = pd.pivot_table(df_away,values=['time_away_count'],index=['orig_tract'],columns=['away_from_home_time'],aggfunc=np.sum,fill_value=0,margins=True)

In [26]:
dff_away.head(20)

time_away_count                                            \
away_from_home_time       1081-1200 1201-1320 121-180 1321-1440 181-240 21-45   
orig_tract                                                                      
36005000100                       0         2       5         1       0     1   
36005000200                      13         4      18        17      10    20   
36005000400                      20        13      20        30      25    31   
36005001600                       9         3      10        12      23    42   
36005001900                       7         7      15        13      22    33   
36005002000                      14        15      31        26      33    36   
36005002300                       6        23       9        20      15    22   
36005002400                       1         2       5         0       0     5   
36005002500                       6         1      20        19      21    50   
36005002701                       7         2       5        13       7    20   
36005002702                       4         8      14        18       9    23   
36005002800                       4         3      17        10      19    40   
36005003100                       7         4      14        13      22     9   
36005003300                       6        11       9         6       9    23   
36005003500                      10         4       9        15      17    22   
36005003700                       0         0       7         4       3     1   
36005003800                       1         1       3         6       4     7   
36005003900                      25        19      50        20      50    49   
36005004001                       2         0       4        11       7    10   
36005004100                      14         3      22        19      20    42   

                                                                           \
away_from_home_time 241-300 301-360 361-420 421-480 46-60 481-540 541-600   
orig_tract                                                                  
36005000100               1       1       0       3     2       1       0   
36005000200               9       5      11      10     8       9       8   
36005000400              16       9      12      18    11      14      16   
36005001600               5       8      12      20    11      26       5   
36005001900               9       8      12       9    20      11       9   
36005002000              19      12      16      17    24      26       7   
36005002300               4      10      14       9     7       4       4   
36005002400               5       2       1       1     2       0       0   
36005002500              14      11       5       8    13       7       8   
36005002701               8       8       3       4     9       5       4   
36005002702               5       5      10      15    11       7       6   
36005002800              11      16      15      11    16      12      11   
36005003100               5       4       5      13     3      10       5   
36005003300               7       4      10       8     7       5       2   
36005003500              11       8      14      10     6      13       5   
36005003700               0       0       1       0     1       0       3   
36005003800               3       1       4       2     4       1       1   
36005003900              29      30      22      23    15      28      29   
36005004001               4       4       4       3     6       2       0   
36005004100              13      21      10      16    20      12      15   

                                                                                
away_from_home_time 601-660 61-120 661-720 721-840 841-960 961-1080  <20   All  
orig_tract                                                                      
36005000100               0      1       0       1       0        0   14    33  
36005000200               2     20      11       5      14  

In [28]:
dff_away.columns = dff_away.columns.droplevel()

In [29]:
dff_away.head(20)

away_from_home_time,1081-1200,1201-1320,121-180,1321-1440,181-240,21-45,241-300,301-360,361-420,421-480,46-60,481-540,541-600,601-660,61-120,661-720,721-840,841-960,961-1080,<20,All
orig_tract,,,,,,,,,,,,,,,,,,,,,
36005000100,0,2,5,1,0,1,1,1,0,3,2,1,0,0,1,0,1,0,0,14,33
36005000200,13,4,18,17,10,20,9,5,11,10,8,9,8,2,20,11,5,14,2,334,530
36005000400,20,13,20,30,25,31,16,9,12,18,11,14,16,7,37,5,15,3,5,580,887
36005001600,9,3,10,12,23,42,5,8,12,20,11,26,5,13,21,5,8,8,8,383,632
36005001900,7,7,15,13,22,33,9,8,12,9,20,11,9,9,22,4,12,7,9,341,579
36005002000,14,15,31,26,33,36,19,12,16,17,24,26,7,9,47,6,13,12,10,714,1087
36005002300,6,23,9,20,15,22,4,10,14,9,7,4,4,6,27,12,9,12,12,331,556
36005002400,1,2,5,0,0,5,5,2,1,1,2,0,0,0,1,0,0,0,0,14,39
36005002500,6,1,20,19,21,50,14,11,5,8,13,7,8,7,25,5,2,3,7,378,610


In [30]:
dff_away.columns

Index(['1081-1200', '1201-1320', '121-180', '1321-1440', '181-240', '21-45',
       '241-300', '301-360', '361-420', '421-480', '46-60', '481-540',
       '541-600', '601-660', '61-120', '661-720', '721-840', '841-960',
       '961-1080', '<20', 'All'],
      dtype='object', name='away_from_home_time')